<a href="https://colab.research.google.com/github/m0tchy/camera-geometry-tutorial/blob/main/03_2D-affine-transform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from numpy import linalg
# from scipy import linalg

import matplotlib.pyplot as plt
from matplotlib import ticker

# プロットのスタイルを同じにするための便利関数
def set_plot_style(ax):
    ax.axis("equal")
    ax.grid(True)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(0.25))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(0.25))


# 回転の逆変換は転置

原点周りで、 x 軸から y 軸へ向かう方向の、角度 $\theta$ の回転を表す変換行列は
$$ R(\theta) := \begin{bmatrix} \cos \theta & -\sin \theta \\ \sin \theta & \cos \theta \end{bmatrix}$$
と書ける.
第1列は $[1, 0]^\top$ の変換先, 第2列は $[0, 1]^\top$ の変換先であることを知っていると覚えやすい。


逆回転はその逆行列 $R(\theta)^{-1}$ であるが,
これは当然 $R(-\theta)$ と等しい.
一方, 計算するとわかる通り, $R(-\theta) = R(\theta)^\top$ となる.

一般に逆行列に関して $A^{-1} A = A A^{-1} = I$ ($I$ は単位行列) が成り立つ.
そうすると、 $A^{-1} = A^\top$ から $A^\top A = AA^\top = I$ が成り立つ.
これは $A$ の各行・各列がお互いに直交していることを表現している。
これが成り立つ $A$ を**直交行列** (orthogonal matrix) という.

回転行列は面積を変えない変換である.
これは $\det R(\theta) = 1$ であることから分かる.

まとめると, 回転行列とは直交行列かつ行列式が 1 であるものと云うことができる.
このような行列を**特殊直交群** (special orthogonal group) といい, 2次元の場合は $\mathrm{SO}(2)$ と表記する. (回転群と呼ぶこともある.)

ちなみに, 直交行列だが行列式が $1$ でないものは鏡映変換（と、さらに回転が含まれるもの）で、行列式は $-1$ になる.
直交行列の行列式は $1$ か $-1$ にしかならない.

In [ ]:
# 回転行列を作る関数
def rotation_matrix(angle_degree):
    ang = np.radians(angle_degree)
    return np.array([
        [np.cos(ang), -np.sin(ang)],
        [np.sin(ang), np.cos(ang)]
    ])

# テスト
R = rotation_matrix(30)
assert np.allclose(R @ R.T, np.identity(2))
assert np.isclose(linalg.det(R), 1.0)

In [ ]:
# 基本ベクトルの回転

X = np.array([[1, 0], [0, 1]]).T
R = rotation_matrix(15)

RX = R @ X

fig, ax = plt.subplots()
set_plot_style(ax)

ax.scatter(X[0], X[1])
ax.scatter(RX[0], RX[1])

for i in range(X.shape[1]):
    ax.arrow(0, 0, X[0][i], X[1][i], length_includes_head=True, head_width=0.1, color="GRAY")
    ax.arrow(0, 0, RX[0][i], RX[1][i], length_includes_head=True, head_width=0.1, color="RED")



# 平面上の線形変換+平行移動 = アフィン変換

平行移動 $\boldsymbol{b}$ が線形変換の後に加えられる変換を**アフィン変換** (affine transform) という.

$$ \boldsymbol{x}' = A\boldsymbol{x} + \boldsymbol{b} $$

複数の点集合を 1 つの行列 
$ X = \begin{bmatrix}\boldsymbol{x}_{0} & \boldsymbol{x}_{1} & \cdots & \boldsymbol{x}_{n-1} \end{bmatrix} \in \mathbb{R}^{2 \times n}$ 
で表すと,
変換後の点集合 
$ X' = \begin{bmatrix} \boldsymbol{x}'_0 & \boldsymbol{x}'_1 & \cdots & \boldsymbol{x}'_{n-1} \end{bmatrix}$ は
$$ X' = AX + B, 
\text{ where } B := \begin{bmatrix}\boldsymbol{b} & \boldsymbol{b} & \cdots & \boldsymbol{b} \end{bmatrix} $$
のようにまとめて書くことができる.

平行移動成分を加える処理は, 平行移動ベクトルを複製して足す必要があり, 数学的には処理が煩雑で美しくない.
ただし NumPy ではブロードキャストという機能のおかげで,
単にベクトルを足すコードで実現することができる.

## 多角形の頂点で観察する

In [ ]:
# 平面上の点集合
# 書くときは、点の座標を横に並べて書くほうが簡単だが、
# 数学で扱う場合、行列としては各列に座標を並べたいので、最後に転置する
# なお、機械学習などでは各行にデータのベクトルを並べたものを扱うほうが普通

X = np.array([
    [0.0, 0.0],
    [0.0, 1.0],
    [1.0, 1.0],
    [1.0, 0.0],
    [0.3, 0.5]
]).T

# 各 "列" が座標
X

In [ ]:
# 線形変換の表現行列
A = rotation_matrix(15)

# 平行移動ベクトル
b = np.array([0.5, 0.2])

In [ ]:
# A, b による変換
# b を縦ベクトル (d x 1 行列) とすることで、
# [b b ... b] のように自動的に増やしてから加算される（ブロードキャスト）

X2 = A @ X + b.reshape(-1, 1)
X2

# ちなみに, 試してみるとわかるが,
# 1D 配列は転置 (.T) をしても効果がない.
# X と同じように転置を使うなら,
# np.array([[0.5, 0.2]]).T のように
# 2D 配列としておく必要がある.

In [ ]:
fig, ax = plt.subplots()
set_plot_style(ax)

ax.fill(X[0], X[1], facecolor=(0,0,0,0.1), edgecolor=(0,0,0,0.5))
ax.fill(X2[0], X2[1], facecolor=(1,0,0,0.1), edgecolor=(1,0,0,0.5))

# 原点の移動
ax.arrow(0, 0, b[0], b[1], head_width=0.05, length_includes_head=True)



## 同次座標を使ったアフィン変換行列

$$ \boldsymbol{x}' = A\boldsymbol{x} + \boldsymbol{b} $$
は, $\boldsymbol{x}$ の**同次座標** (homogeneous coordinates) $[ \boldsymbol{x}^\top \, 1]^\top$ を使うと,
$$ \boldsymbol{x}' = \begin{bmatrix}A & \boldsymbol{b} \end{bmatrix} \begin{bmatrix} \boldsymbol{x} \\ 1 \end{bmatrix} $$
と書くことができる.

In [ ]:
# X に 1 を追加して, 同次座標にする.

# 確認: 点の個数だけ 1 を作る方法
# 1 x n の配列にする ( n == X.shape[1] )
print(np.ones( (1, X.shape[1]) ))

# 行として結合
X_ = np.r_[X, np.ones((1, X.shape[1]))]
X_

In [ ]:
# アフィン変換行列
# A と b を列方向に結合
Ab = np.c_[A, b]
Ab

In [ ]:
# [ A, b ] による変換

X3 = Ab @ X_
assert np.array_equal(X2, X3)

X3

In [ ]:
# X3 をプロットすれば、 X2 と同じなるのは当たり前なので省略.
# 各自でやってください.

## 結果も同次座標にする

$[A\; \boldsymbol{b}]$ は同次座標に対して掛け算するだけで線形変換と平行移動をすることができるが,
結果が普通の座標となるので, 
連続して変換を計算することができない。

そこで、変換の結果が自動的に同次座標になるようにする。
変換前の点の同次座標を $\begin{bmatrix} \boldsymbol{x}^\top & 1 \end{bmatrix}^\top$,
変換後の点の同次座標を $\begin{bmatrix} \boldsymbol{x}'^\top & 1 \end{bmatrix}^\top$ とすると,
$$ \begin{bmatrix} \boldsymbol{x}' \\ 1 \end{bmatrix} = \begin{bmatrix} A & \boldsymbol{b} \\ \boldsymbol{0}_2^\top & 1 \end{bmatrix} \begin{bmatrix} \boldsymbol{x} \\ 1 \end{bmatrix} $$
と書くことができる。

In [ ]:
# 同次化されたアフィン変換行列を作る
Ab_ = np.r_[ np.c_[A, b], [[0, 0, 1]] ]
Ab_

# numpy 配列で複数の値を組み合わせて新しい配列を作る方法はいろいろある。
# 例えば、 np.block なども利用できる
np.block([[A, b.reshape(-1,1)],[0,0,1]])

In [ ]:
# アフィン変換を適用する
X4_ = Ab_ @ X_

# さらに変換させてみる
X5_ = Ab_ @ X4_

print(X4_)
print(X5_)

In [ ]:
# 同次座標はそのままプロットに渡せない
# 第3成分を除く必要がある
X4 = X4_[:-1]
X5 = X5_[:-1]

fig, ax = plt.subplots()
set_plot_style(ax)

ax.fill(X[0], X[1], facecolor=(0,0,0,0.1), edgecolor=(0,0,0,0.5))
ax.fill(X4[0], X4[1], facecolor=(1,0,0,0.1), edgecolor=(1,0,0,0.5))
ax.fill(X5[0], X5[1], facecolor=(0,0,1,0.1), edgecolor=(0,0,1,0.5))

# 練習問題

## Q1
最後の例を、 for ループを使って 10 回程度連続で行い、描画する（色はすべて同じでよい）


## Q2

`rotation_matrix(deg)` の同次化版 `rotation_matrix_h(deg)` を定義する。同じように平行移動変換 `translation_matrix_h(tx, ty)`, スケーリング `scaling_matrix_h(sx, sy)` を定義してテストする


## Q3

指定した点 $\boldsymbol{x}$ を中心に回転させる変換は, 回転を $R(\theta)$,
平行移動を $T(\boldsymbol{x})$ とすると, $Q(\theta, x) = T(\boldsymbol{x})R(\theta)T(-\boldsymbol{x})$ と書ける。3つの変換を1つずつ順番に描画して、回転が正しく行われていることを確認しなさい。

## 解答例

最初は見ないでやってみること。

In [ ]:
# Q1 
fig, ax = plt.subplots()
set_plot_style(ax)

ax.fill(X[0], X[1], facecolor=(0,0,0,0.1), edgecolor=(0,0,0,0.5))

XX_ = X_
for i in range(20):
    XX_ = Ab_ @ XX_
    XX = XX_[:-1]
    ax.fill(XX[0], XX[1], facecolor=(0,0,0,0.1), edgecolor=(0,0,0,0.5))


In [ ]:
# Q2

def rotation_matrix_h(angle_degree):
    ang = np.radians(angle_degree)
    return np.array([
        [np.cos(ang), -np.sin(ang), 0],
        [np.sin(ang), np.cos(ang), 0],
        [0, 0, 1]
    ])

def translation_matrix_h(tx, ty):
    return np.array([
        [1, 0, tx],
        [0, 1, ty],
        [0, 0, 1]
    ])

def scaling_matrix_h(sx, sy):
    return np.array([
        [sx, 0, 0],
        [0, sy, 0],
        [0, 0, 1]
    ])



In [ ]:
# Q3

p = (0.3, 0.5)
ang_deg = 15

X1_ = translation_matrix_h(-p[0], -p[1]) @ X_
X2_ = rotation_matrix_h(ang_deg) @ X1_
X3_ = translation_matrix_h(p[0], p[1]) @ X2_

fig, ax = plt.subplots()
set_plot_style(ax)

ax.fill(X[0], X[1], facecolor=(1,0,0,0.1), edgecolor=(1,0,0,0.5))
ax.fill(X1_[:-1][0], X1_[:-1][1], facecolor=(0,0,0,0.1), edgecolor=(0,0,0,0.5))
ax.fill(X2_[:-1][0], X2_[:-1][1], facecolor=(0,0,0,0.1), edgecolor=(0,0,0,0.5))
ax.fill(X3_[:-1][0], X3_[:-1][1], facecolor=(0,0,1,0.1), edgecolor=(0,0,1,0.5))
